In [1]:
# 本章で使うモジュールのインポート
from torch import utils
from torchvision import datasets
import torchvision.transforms as transforms

import torch
from torch import nn, optim
from torch.nn import functional as F

In [2]:
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())  # 学習用データセット
train_loader = utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)  # ミニバッチごとにデータを纏める(学習時にはshuffle=True)

testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())  # 検証用データセット
test_loader = utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)  # ミニバッチごとにデータを纏める(学習時にはshuffle=False)

In [6]:
class MlpNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 10)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters())

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [7]:
def train(model, train_loader):
    model.train()
    
    total_correct = 0
    total_loss = 0
    total_data_len = 0    
    
    for batch_imgs, batch_labels in train_loader:
        # Flatten the batch of images
        batch_imgs = batch_imgs.reshape(-1, 28*28*1)
        labels = torch.eye(10)[batch_labels]
        
        # Zero the parameter gradients
        model.optimizer.zero_grad()
        # Forward pass
        outputs = model(batch_imgs)
        # Compute the loss
        loss = model.criterion(outputs, labels)
        # Backward pass and optimize
        loss.backward()
        model.optimizer.step()
        
        _, pred_labels = torch.max(outputs, axis=1)
        batch_size = len(batch_labels)
        for i in range(batch_size):
            total_data_len += 1
            if pred_labels[i] == batch_labels[i]:
                total_correct += 1
        total_loss += loss.item()
        
    accuracy = total_correct / total_data_len * 100
    loss = total_loss / total_data_len
    
    return accuracy, loss
        

In [8]:
model = MlpNet()
accracy, loss = train(model, train_loader)
print(f"{accracy=}, {loss=}")

accracy=94.58166666666666, loss=0.00015825869853142648


In [11]:
class MlpNet2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 10)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=0.01)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [12]:
model2 = MlpNet2()
accracy, loss = train(model2, train_loader)
print(f"{accracy=}, {loss=}")

accracy=84.23166666666667, loss=0.0003574816721336295
